In [1]:
import pandas as pd
import geopandas as gpd
# get the census data for nyc and filiter it for just the most recent year
census = gpd.read_file("https://data.mixi.nyc/census.geojson")
data = census[census.year == census.year.max()]
# get just the data we need
data = data[data.median_inc > 0][[
    "geoid", "name", "year", "median_inc", "geometry"]]
# scale the income from 0..1 as a percent of the max income in our data set
max_income = data.median_inc.max()
min_income = data.median_inc.min()
data["scaled_income"] = data.median_inc.apply(
    lambda x: (x - min_income) / (max_income - min_income))
data

,geoid,name,year,median_inc,geometry,scaled_income
24,36061000201,Census Tract 2.01; New York County; New York,2022,45582.0,"POLYGON ((-73.98450 40.70952, -73.98655 40.709...",0.143239
37,36061000600,Census Tract 6; New York County; New York,2022,25655.0,"POLYGON ((-73.99022 40.71441, -73.98934 40.714...",0.059720
50,36061001401,Census Tract 14.01; New York County; New York,2022,89873.0,"POLYGON ((-73.98837 40.71645, -73.98754 40.716...",0.328871
63,36061001402,Census Tract 14.02; New York County; New York,2022,46615.0,"POLYGON ((-73.98507 40.71909, -73.98423 40.718...",0.147568
76,36061001800,Census Tract 18; New York County; New York,2022,74485.0,"POLYGON ((-73.98986 40.72053, -73.98972 40.720...",0.264377
...,...,...,...,...,...,...
27245,36005003100,Census Tract 31; Bronx County; New York,2022,41370.0,"POLYGON ((-73.90447 40.81229, -73.90418 40.812...",0.125585
27258,36005023000,Census Tract 230; Bronx County; New York,2022,55625.0,"POLYGON ((-73.86780 40.85160, -73.86681 40.850...",0.185331
27287,36085024800,Census Tract 248; Richmond County; New York,2022,131146.0,"POLYGON ((-74.22948 40.51909, -74.22963 40.519...",0.501855
27300,36081003400,Census Tract 34; Queens County; New York,2022,79728.0,"POLYGON ((-73.85053 40.68760, -73.84999 40.686...",0.286351


In [3]:
# make a new column called "color" that we can use for the choropleth
from matplotlib.colors import to_hex
from matplotlib import colormaps

# get a "colormap" scale
# see: https://matplotlib.org/stable/users/explain/colors/colormaps.html
cmap_name = 'PiYG'
cmap = colormaps[cmap_name]
# create a function to map income `x` to a hex color code


def map_income(x):
    color = cmap(x)
    color = to_hex(color)
    return color


data["color"] = data.scaled_income.apply(map_income)

# preview the map in python
# data.explore(column="scaled_income", cmap=cmap_name, legend=True)
data.set_crs(epsg=4326).to_file("income.geojson", driver="GeoJSON")

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [4]:
out = data[['geoid', 'name', 'median_inc', 'scaled_income', 'color']]
out.to_csv("data.csv", index=False)
out.to_file("choro.geojson", driver="GeoJSON")

In [12]:
choro = pd.read_csv("data.csv")
choro.drop(columns=['geometry'], inplace=True)
choro.columns

Index(['name', 'median_inc', 'scaled_income', 'color'], dtype='object')